# SQL Investigation
1. Run all cells.
1. View report at the bottom.

In [1]:
# These are just defaults will be overwritten if you use nimport pip
db = "Tfs_tfsprodcus2_37253a68-972a-4bf4-8c5f-a259ba4d42cd"
start = "2019-07-31T17:30:00.0000000Z"
end = "2019-07-31T18:30:36.0000000Z"
url = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"

In [ ]:
# This isn't needed if you are bootstraping
#!pip install Kqlmagic --no-cache-dir --upgrade
#!pip install nimport --no-cache-dir --upgrade
#!pip install azure-kusto-notebooks --no-cache-dir --upgrade

In [2]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd
pd.options.display.html.table_schema = True
from pandas import Series, DataFrame
from datetime import datetime, timedelta, timezone
from urllib.parse import urlencode, quote_plus
from requests.utils import requote_uri
import time
import numpy as np
from matplotlib import pyplot as plt
from nimport.utils import tokenize, open_nb
import json
import os
import calendar as cal
import concurrent.futures
from azure.kusto.notebooks import utils as akn

In [3]:
params = {
    "db": db,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
    

In [4]:
# todo: use from akn
def to_datetime(timestamp):
    s = timestamp[:23] + 'Z' # only allow 5 decimals of precision
    for f in ("%Y-%m-%d %H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%S.%fZ"):
        try:
            return datetime.strptime(s, f)
        except:
            pass
def get_time(timestamp, d):
    return int((cal.timegm(to_datetime(timestamp).timetuple()) + (d * 60)) * 1000)
# end todo

def getKustoQuery(csl_filename, params):

    return tokenize(os.path.join(queryPath, csl_filename), params)

queryPath = os.path.join(root, 'queries')

states = {
    "ja": "ja",
    "healthagent": "healthagent"
};

state = "";

In [5]:
%%capture
%load_ext Kqlmagic

In [6]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FQPTGUJSM to authenticate.


In [32]:
sqlPath = os.path.join(queryPath, 'sql')
q_data = os.path.join(sqlPath, "GetData.csl")
q_whatsSlow = os.path.join(sqlPath, "WhatsSlow.csl")
with concurrent.futures.ThreadPoolExecutor() as executor:
    # materialize so that we have all information we might need
    p1 = executor.submit(akn.execute_file, client, 'VSO', q_data, params)
    q_data_df = akn.to_dataframe_from_future(p1)
    params["service"] = q_data_df["Service"][0]
    params["su"] =q_data_df["ScaleUnit"][0]
    
    p2 = executor.submit(akn.execute_file, client, 'VSO', q_whatsSlow, params)

q_whatsSlow_df = akn.to_dataframe_from_future(p2)  


In [81]:
# Initialize for further analysis later
q_cpuTop_df = None
q_cpuXEvent_df = None

In [80]:
def cpuAnalysis():
    global q_cpuTop_df
    global q_cpuXEvent_df
    q_cpuTop = os.path.join(sqlPath, "CpuTop.csl")
    q_cpuXEvent = os.path.join(sqlPath, "CpuXevent.csl")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        p1 = executor.submit(akn.execute_file, client, 'VSO', q_cpuTop, params)
        p2 = executor.submit(akn.execute_file, client, 'VSO', q_cpuXEvent, params)

    q_cpuTop_df = akn.to_dataframe_from_future(p1)  
    q_cpuXEvent_df = akn.to_dataframe_from_future(p2)


In [84]:
print('=' * 50)
print('Report!')
print('=' * 50, '\n\n')

jarvisParams = {'su': params["su"], 'start': get_time(start, -10), 'end': get_time(end, 10), 'service': params["service"], 'db': db }

jaJarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/PlatformViews/SQLAzureDatabase""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"},""" \
    """{"query":"//*[id='__DatabaseName']","key":"value","replacement":"%(db)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
print('Jarvis dashboard link for sql:\n', requote_uri(jaJarvisLink), '\n')

print()

## Where is the database at?
print("Database is at: ")
so = q_whatsSlow_df["ServiceObjective"].unique()
if so.size > 1:
    print("We found different service objectives..looks like db was changed?")
print(so) 

print()

## What's slow?
cpu = q_whatsSlow_df["avg_AverageCpuPercentage"]
memory = q_whatsSlow_df["avg_AverageMemoryUsagePercentage"]
logWrite= q_whatsSlow_df["avg_AverageLogWriteUtilizationPercentage"]
worker= q_whatsSlow_df["max_MaximumWorkerPercentage"]
cpu_coefficientOfVariance = cpu.std()/cpu.mean()
memory_coefficientOfVariance = memory.std()/memory.mean()
logWrite_coefficientOfVariance = logWrite.std()/logWrite.mean()
worker_coefficientOfVariance = worker.std()/worker.mean()
maxVar = 0.5

reasons = "Possibly due to: "
if cpu_coefficientOfVariance >= maxVar:
    reasons+= "cpu (max: %s), " % (cpu.max())
if memory_coefficientOfVariance >= maxVar:
    reasons+= "memory (max: %s), " % (memory.max())
if logWrite_coefficientOfVariance >= maxVar:
    reasons+= "logwrite (max: %s), " % (logWrite.max())
if worker_coefficientOfVariance >= maxVar:
    reasons+= "worker (max: %s), " % (worker.max())
print(reasons)

if cpu.max() >= 80:
    print("We found high CPU, let's start with CPU analysis...")
    
    cpuAnalysis()
    
    print()
    print("Top CPU commands:")
    display(q_cpuTop_df)
    
    print()
    print("Who's causing these commands?:")
    display(q_cpuXEvent_df)



Report!


Jarvis dashboard link for sql:
 https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/PlatformViews/SQLAzureDatabase?overrides=[%7B%22query%22:%22//*[id='Service']%22,%22key%22:%22value%22,%22replacement%22:%22tfs%22%7D,%7B%22query%22:%22//*[id='ScaleUnit']%22,%22key%22:%22value%22,%22replacement%22:%22tfs-cus-2%22%7D,%7B%22query%22:%22//*[id='__DatabaseName']%22,%22key%22:%22value%22,%22replacement%22:%22Tfs_tfsprodcus2_37253a68-972a-4bf4-8c5f-a259ba4d42cd%22%7D]&globalStartTime=1564593600000&globalEndTime=1564598436000&pinGlobalTimeRange=true 


Database is at: 
['BC_Gen5_10']

Possibly due to: cpu (max: 99.35499999999999), logwrite (max: 10.2275), worker (max: 10.6), 
We found high CPU, let's start with CPU analysis...

Top CPU commands:


,QueryText,sum_TotalCpuTime,sum_TotalPhysicalReads,sum_TotalLogicalReads,sum_TotalExecutions,sum_TotalExceptions
0,"(@partitionId int,@projectId uniqueidentifier)...",1938592,5,645495201,55190,0
1,"SELECT TOP 20001 Results.SourceId, Results.Tar...",516171,137,403243486,23,0
2,"(@p0 datetime,@p1 nvarchar(4000))SELECT TOP 11...",429585,4504,135753048,81,0
3,IF (SESSION_CONTEXT(N'AnonymousAccess') IS NUL...,385454,0,174,6179536,0
4,"(@partitionId int)SELECT W.Id,\r\n ...",384675,120732,204499708,31235,0
5,"(@partitionId int,@changeSetId int,@lastServer...",306695,40946,119146317,2602,0
6,"(@partitionId int,@projectId int,@workItemType...",300427,17414,15860059,3881,0
7,"(@partitionId int,@dataspaceId int,@repository...",266177,82,2535002,56797,0
8,"SELECT TOP 20001 lhs.Id AS Id, CONVERT(INT, SU...",206388,1,203975387,7,0
9,"(@partitionId int,@workspaceId int,@serverPath...",191289,150287,6083731,3389,0



Who's causing these commands?:


,TypeName,ObjectName,sum_CpuTime
0,Activity,,1886346
1,Other,prc_ProjectGet,1572332
2,Job,Build.prc_GetDefinitions,1086682
3,Activity,prc_GetWorkItemFieldValues,419301
4,Other,prc_DispatchMessageQueues,322856
5,Job,prc_ProjectGet,290591
6,Other,prc_SetMessageQueuesOffline,258450
7,Activity,prc_GitRepositoryIdFromName,228428
8,Activity,GetMetadataTimestamps,205928
9,PipelineActivity,prc_AddContainerItems,205414
